In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install correct PyTorch version (2.0.1) and dependencies
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 544.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=5aecf710750bdaaeb14a0e0330655c918eea1168a353670d4989fd5d2b6aa27b
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0

In [5]:
# Install PyTorch Geometric + extensions (scatter, sparse, etc.)
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00


In [3]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 48.7 MB/s eta 0:00:00


In [6]:
#setting up paths

import os
import torch
import pandas as pd
from tqdm import tqdm
from torch_geometric.data import Data

#Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {device}")

#Define paths
project_root = "/content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA"
data_path = f"{project_root}/data"


Using Device: cpu



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [7]:
#Creating drug graphs
#Setting up environment

from rdkit import Chem
from rdkit.Chem import rdmolops
from torch_geometric.data import Data
import torch
from tqdm import tqdm

#Atom feature encoder
atom_types = ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I', 'H', 'Si', 'Mg', 'Na', 'Ca', 'Fe', 'As', 'Al', 'B', 'V', 'K', 'Tl', 'Sb', 'Sn', 'Ag', 'Co', 'Se', 'Ti', 'Zn', 'Li', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Unknown']

#One-hot encoder wth fallback
def one_hot_encoding(x, allowable_set):
  encoding = [int(x == s) for s in allowable_set]
  if sum(encoding) == 0:
    encoding.append(1)
  else:
    encoding.append(0)
  return encoding


def atom_to_feature_vector(atom):
  return (one_hot_encoding(x=atom.GetSymbol(), allowable_set=atom_types) +              # 36
          one_hot_encoding(atom.GetDegree(), [0,1,2,3,4,5,6,7]) +                       # 8
          one_hot_encoding(atom.GetTotalNumHs(), [0,1,2,3,4,5,6,7,8,9,10]) +            # 11
          one_hot_encoding(atom.GetTotalValence(), [0,1,2,3,4,5,6,7,8,9,10]) +          # 11
          one_hot_encoding(atom.GetIsAromatic(), [0,1]) +                               # 2
          one_hot_encoding(atom.GetChiralTag(), [0,1,2,3,4,5,6,7,8]))                   # 9 = 77

#SMILES to PyG graph
def mol_to_graph(smiles):
  mol = Chem.MolFromSmiles(smiles)
  if mol is None:
    return None

  node_feats = []
  for atom in mol.GetAtoms():
    node_feats.append(atom_to_feature_vector(atom))

  edge_index = []
  for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    edge_index.append([i, j])
    edge_index.append([j, i]) #undirected

  x = torch.tensor(node_feats, dtype=torch.float)
  edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

  return Data(x=x, edge_index=edge_index)

#Load KIBA Dataset
kiba_df = pd.read_csv(f"{data_path}/kiba_affinity_df.csv")

#Build unique drug graphs
drug_graphs = {}

for _, row in tqdm(kiba_df.iterrows(), total=len(kiba_df)):
  drug_idx = row['Drug_Index']
  smiles = row['SMILES']
  if drug_idx not in drug_graphs:
    graph = mol_to_graph(smiles)
    if graph:
      drug_graphs[drug_idx] = graph

#Save drug graphs
torch.save(drug_graphs, f"{data_path}/kiba_drug_graphs.pt")
print(f"Saved {len(drug_graphs)} unique drug graphs.")

100%|██████████| 118254/118254 [00:12<00:00, 9740.91it/s] 


Saved 2111 unique drug graphs.


In [9]:
#Check to verify graphs were saved correctly
loaded_graphs = torch.load(f"{data_path}/kiba_drug_graphs.pt")
print(f"Example Graph:\n{loaded_graphs[0]}")

Example Graph:
Data(x=[32, 83], edge_index=[2, 70])
